In [1]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
save_dir = '/data/speech/digital_human/preprocessed/'
!mkdir $save_dir

In [9]:
import pandas as pd

def convert_to_seconds(time_str):
    time_parts = list(map(float, time_str.split(':')))
    hours = time_parts[0] * 3600  # 1시간은 3600초입니다.
    minutes = time_parts[1] * 60  # 1분은 60초입니다.
    seconds = time_parts[2]
    frames = time_parts[3] / 60  # FPS가 60이므로, 각 프레임은 1/60초를 나타냅니다.

    total_seconds = hours + minutes + seconds + frames
    return total_seconds

def read_csv(csv_file):
    df = pd.read_csv(csv_file)
    df['Seconds'] = df['Timecode'].apply(convert_to_seconds)
    df['Seconds'] = df['Seconds'].apply(lambda x: x - df['Seconds'][0])
    if 'Timecode' in df.columns:
        df = df.drop(columns=['Timecode'])
    if 'BlendShapeCount' in df.columns:    
        df = df.drop(columns=['BlendShapeCount'])
    if 'BlendshapeCount' in df.columns:
        df = df.drop(columns=['BlendshapeCount'])
    ndf = pd.DataFrame(columns=df.columns)
    i = 0
    prev_j = 0
    while True:
        s = i / 30
        finded = False
        for j in range(prev_j, len(df)-2):
            ls = df['Seconds'][j]
            rs = df['Seconds'][j+1]
            if s >= ls and s < rs:
                prev_j = j
                v = df.iloc[j]
                finded = True
                break

        if finded:
            ndf.loc[len(ndf)] = v
        else:
            break

        i += 1
    return ndf

In [10]:
import os

root_dir = '/data/speech/DigitalHuman/ml'

def find_pair_files(root_dir):
    data = []
    files = [os.path.join(root_dir, file) for file in os.listdir(root_dir)]
    pair_file = [None, None]
    for file in files:
        if os.path.isdir(file):
            d = find_pair_files(file)
            if len(d) > 0:
                data.extend(d)
        if file.endswith('_raw.csv'):
            pair_file[0] = os.path.join(root_dir, file)
        if file.endswith('.mov'):
            pair_file[1] = os.path.join(root_dir, file)
    if pair_file[0] != None and pair_file[1] != None:
        data.append(pair_file)
    return data
            
pair_files = find_pair_files(root_dir)
print(len(pair_files))

86


In [11]:
data_dict = {}
for csv, mov in pair_files:
    name = csv.split('/')[5] 
    if name not in data_dict:
        data_dict[name] = []
    data_dict[name].append([csv, mov])

print(data_dict.keys())

for key in data_dict:
    dir = os.path.join(save_dir, key)
    if not os.path.exists(dir):
        os.mkdir(dir)
    
!ls -lt $save_dir

dict_keys(['kyuchulLee', 'soochulPark', 'jeewonPark', 'nohsikPark', 'jinwooOh', 'yehunHwang', 'kyuseokKim'])
total 28
drwxrwxr-x 2 scpark scpark 4096  8월  7 21:23 jeewonPark
drwxrwxr-x 2 scpark scpark 4096  8월  7 21:23 jinwooOh
drwxrwxr-x 2 scpark scpark 4096  8월  7 21:23 kyuseokKim
drwxrwxr-x 2 scpark scpark 4096  8월  7 21:23 nohsikPark
drwxrwxr-x 2 scpark scpark 4096  8월  7 21:23 soochulPark
drwxrwxr-x 2 scpark scpark 4096  8월  7 21:23 yehunHwang
drwxrwxr-x 2 scpark scpark 4096  8월  7 21:23 kyuchulLee


In [ ]:
import numpy as np
import librosa
from multiprocessing.dummy import Pool

def f(params):
    try:
        key, csv_file, mov_file = params
        print(csv_file)
        save_file = os.path.join(save_dir, key, os.path.basename(csv_file[:-4]))
        if os.path.exists(save_file + '.npy'):
            return
        csv = read_csv(csv_file)
        blendshapes = np.array(csv)[:, :61]
        wav, _ = librosa.load(mov_file, sr=44100)
        data = {'wav': wav,
                'blendshapes': blendshapes}
        np.save(save_file, data)
    except:
        return
    
for key in data_dict:
    print(key)
    f_list = [(key, pair_file[0], pair_file[1]) for pair_file in data_dict[key]]
    with Pool(8) as p:
        p.map(f, f_list)

kyuchulLee
/data/speech/DigitalHuman/ml/kyuchulLee/data/iphone/20230626_MH_ARKit_006_4/MH_ARKit_006_4_iPhone_raw.csv
/data/speech/DigitalHuman/ml/kyuchulLee/data/iphone/20230626_MH_ARKit_006_3/MH_ARKit_006_3_iPhone_raw.csv
/data/speech/DigitalHuman/ml/kyuchulLee/data/iphone/20230626_MH_ARKit_006_7/MH_ARKit_006_7_iPhone_raw.csv
/data/speech/DigitalHuman/ml/kyuchulLee/data/iphone/20230626_MH_ARKit_006_9/MH_ARKit_006_9_iPhone_raw.csv
/data/speech/DigitalHuman/ml/kyuchulLee/data/iphone/20230626_MH_ARKit_006_5/MH_ARKit_006_5_iPhone_raw.csv
/data/speech/DigitalHuman/ml/kyuchulLee/data/iphone/20230626_MH_ARKit_006_6/MH_ARKit_006_6_iPhone_raw.csv
/data/speech/DigitalHuman/ml/kyuchulLee/data/iphone/20230626_MH_ARKit_006_10/MH_ARKit_006_10_iPhone_raw.csv
/data/speech/DigitalHuman/ml/kyuchulLee/data/iphone/20230626_MH_ARKit_006_1/MH_ARKit_006_1_iPhone_raw.csv
/data/speech/DigitalHuman/ml/kyuchulLee/data/iphone/20230626_MH_ARKit_006_8/MH_ARKit_006_8_iPhone_raw.csv
/data/speech/DigitalHuman/ml/kyuc

In [ ]:
print('done')